
# Risk Prediction - Ouray County Parcvel Risk
**Author:** Bryce A Young  
**Created:** 2025-01-17 | 
**Modified:** 2025-01-17  

#### Overview
This notebook trains and tests various supervized machine learning models to predict home risk scores. For each model, we explore and tune the parameters to minimize loss, and we analyze the suitability of each model.